In [1]:
#Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv  

# Load the image
image = cv.imread('../images/the_berry_farms_sunflower_field.jpeg', cv.IMREAD_REDUCED_COLOR_4)
gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
assert gray_image is not None, 'ERROR!!! Image not found'

# Define a function to create a Gaussian kernel
def GaussianKernal(size, sigma, show_plot=False):
    oneDKernal = np.linspace(-(size // 2), size // 2, size)
    oneDKernal = np.exp(-(oneDKernal**2) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))
    twoDKernal = np.outer(oneDKernal.T, oneDKernal.T)
    twoDKernal *= 1.0 / twoDKernal.max()

    if show_plot:
        plt.imshow(twoDKernal, interpolation='none', cmap='gray')
        plt.title("Gaussian Kernel")
        plt.show()

    return twoDKernal

sigma_value = 0.64

# Create a Gaussian kernel
gaussian_filter = GaussianKernal(5, sigma_value, show_plot=False)
L_GKernal = cv.Laplacian(gaussian_filter, cv.CV_64F) * sigma_value**2           # Get the Laplacian of the Gaussian kernel
L_Gimage = cv.filter2D(gray_image, -1, L_GKernal)                               # Applying Laplacian of Gaussian kernel to the image
Threshold = 81                                                                  # Setting a Thresholdi(Adjustable)
_, binary_image = cv.threshold(np.abs(L_Gimage), Threshold, 255, cv.THRESH_BINARY)

# Get Contours
contours, _ = cv.findContours(binary_image.astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for i in contours:                                                        # Mark Circles in the original image
    ((x, y), Radius) = cv.minEnclosingCircle(i)                           # Getting the center and radius of the blob
    center_coordinates = (int(x), int(y))
    Radius = int(Radius)
    cv.circle(image, center_coordinates, Radius, (255, 0, 255), 2)        # magenta coloured circles with 2 thickness

image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)                          # Convert BGR image to RGB for Matplotlib display

# Display Results
plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
plt.imshow(L_Gimage, interpolation='none', cmap='gray')
plt.title("Laplacian of Gaussian")
plt.subplot(1, 3, 2)
plt.imshow(binary_image, interpolation='none', cmap='gray')
plt.title("Binary Image")
plt.subplot(1, 3, 3)
plt.imshow(image_rgb, interpolation='none')                               # No cmap needed for RGB
plt.title("Original Image with Blobs")
plt.show()

print("Detected Number of Blobs: {}".format(len(contours)))

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [2]:
#Question 2
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
#Mouse clicking
k = 0
def mousePoint(event, x,y,flags,params):
    if event == cv.EVENT_LBUTTONDOWN:
        imgPoints.append([x,y,1])

def Homography(imgPoints, flagPoints):
    A = []
    for i in range(4):
        A.append([flagPoints[i][0],flagPoints[i][1],flagPoints[i][2],0,0,0,
        -imgPoints[i][0]*flagPoints[i][0],-imgPoints[i][0]*flagPoints[i][1],-imgPoints[i][0]*flagPoints[i][2]])
        A.append([0,0,0,flagPoints[i][0],flagPoints[i][1],flagPoints[i][2],
        -imgPoints[i][1]*flagPoints[i][0],-imgPoints[i][1]*flagPoints[i][1],-imgPoints[i][1]*flagPoints[i][2]])
    A = np.array(A,dtype = np.float64)
    AAT = A.T @ A
    w, v = np.linalg.eig(AAT)
    H = v[:, np.argmin(w)]
    H = H.reshape((3,3))
    H = H/H[-1,-1]
    return H

img = cv.imread('Struc.jpg',cv.IMREAD_ANYCOLOR)
img = cv.resize(img,(600,400))
cv.imshow("IMG",img)
imgPoints = []
cv.setMouseCallback("IMG",mousePoint)
k = cv.waitKey(0)
if k ==ord('q'): 
    cv.destroyAllWindows()
imgFlag = cv.imread('flag.png',cv.IMREAD_ANYCOLOR)
flagheight,flagWidth = imgFlag.shape[0],imgFlag.shape[1]
flagPoints = np.array([[0,0,1],[flagWidth,0,1],[flagWidth,flagheight,1],[0,flagheight,1]])
imgPoints = np.array(imgPoints)
H = Homography(imgPoints,flagPoints)
print('H= ',H)
rows,cols,ch = img.shape
warpedImg = cv.warpPerspective(imgFlag,H,(cols,rows))
overlayImg = cv.add(img, warpedImg)
fig,ax = plt.subplots(1,4,figsize = (30,15))
warpedImg = cv.cvtColor(warpedImg, cv.COLOR_BGR2RGB)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
imgFlag = cv.cvtColor(imgFlag, cv.COLOR_BGR2RGB)
overlayImg = cv.cvtColor(overlayImg, cv.COLOR_BGR2RGB)
ax[0].imshow(img)
ax[0].set_title("structural image")
ax[1].imshow(imgFlag)
ax[1].set_title("Flag")
ax[2].imshow(warpedImg)
ax[2].set_title("Warped Image")
ax[3].imshow(overlayImg)
ax[3].set_title("Overlayed Image")


IndexError: index 1 is out of bounds for axis 0 with size 1

In [4]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

# Initialize variables
click_count = 0

# Define mouse click event handler
def handle_mouse_click(event, x, y, flags, params):
    if event == cv.EVENT_LBUTTONDOWN:
        image_points.append([x, y, 1])

# Calculate Homography matrix
def calculate_homography(image_points, flag_points):
    A_matrix = []
    for i in range(4):
        A_matrix.append([flag_points[i][0], flag_points[i][1], flag_points[i][2], 0, 0, 0,
                          -image_points[i][0] * flag_points[i][0], -image_points[i][0] * flag_points[i][1],
                          -image_points[i][0] * flag_points[i][2]])
        A_matrix.append([0, 0, 0, flag_points[i][0], flag_points[i][1], flag_points[i][2],
                          -image_points[i][1] * flag_points[i][0], -image_points[i][1] * flag_points[i][1],
                          -image_points[i][1] * flag_points[i][2]])

    A_matrix = np.array(A_matrix, dtype=np.float64)
    AAT_matrix = A_matrix.T @ A_matrix
    eigenvalues, eigenvectors = np.linalg.eig(AAT_matrix)
    homography_matrix = eigenvectors[:, np.argmin(eigenvalues)]
    homography_matrix = homography_matrix.reshape((3, 3))
    homography_matrix = homography_matrix / homography_matrix[-1, -1]
    return homography_matrix

# Load and display the image
image = cv.imread('Struc.jpg', cv.IMREAD_ANYCOLOR)
image = cv.resize(image, (600, 400))
cv.imshow("Image", image)
image_points = []
cv.setMouseCallback("Image", handle_mouse_click)
key = cv.waitKey(0)

# Close the window on 'q' key press
if key == ord('q'):
    cv.destroyAllWindows()

# Load the flag image and define flag points
flag_image = cv.imread('flag.png', cv.IMREAD_ANYCOLOR)
flag_height, flag_width = flag_image.shape[0], flag_image.shape[1]
flag_points = np.array([[0, 0, 1], [flag_width, 0, 1], [flag_width, flag_height, 1], [0, flag_height, 1]])
image_points = np.array(image_points)

# Calculate Homography matrix
homography_matrix = calculate_homography(image_points, flag_points)
print('Homography Matrix = ', homography_matrix)

# Warp and overlay images
rows, cols, ch = image.shape
warped_image = cv.warpPerspective(flag_image, homography_matrix, (cols, rows))
overlayed_image = cv.add(image, warped_image)

# Create subplots for displaying images
fig, ax = plt.subplots(1, 4, figsize=(30, 15))
warped_image = cv.cvtColor(warped_image, cv.COLOR_BGR2RGB)
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
flag_image = cv.cvtColor(flag_image, cv.COLOR_BGR2RGB)
overlayed_image = cv.cvtColor(overlayed_image, cv.COLOR_BGR2RGB)
ax[0].imshow(image)
ax[0].set_title("Structural Image")
ax[1].imshow(flag_image)
ax[1].set_title("Flag")
ax[2].imshow(warped_image)
ax[2].set_title("Warped Image")
ax[3].imshow(overlayed_image)
ax[3].set_title("Overlayed Image")

# Display the subplots
plt.show()


IndexError: index 2 is out of bounds for axis 0 with size 2

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

# Function to compute SIFT feature matches
def computeSIFTMatches(image1, image2):
    sift = cv.xfeatures2d.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    points1 = []
    points2 = []

    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7 * n.distance:
            points2.append(keypoints2[m.trainIdx].pt)
            points1.append(keypoints1[m.queryIdx].pt)

    points1 = np.int32(points1)
    points2 = np.int32(points2)
    src = np.concatenate((points1, np.ones((points1.shape[0], 1))), axis=1)
    dst = np.concatenate((points2, np.ones((points2.shape[0], 1))), axis=1)
    return src, dst

# Function to compute RANSAC homography
def computeRANSACHomography(src, dst):
    numInliers = 0
    inlierSrc = []
    inlierDst = []
    
    for _ in range(100):
        # Generate 4 random indices for choosing 4 points
        randIndices = np.random.randint(0, src.shape[0], 4)
        
        while len(set(randIndices)) < 4:
            randIndices = np.random.randint(0, src.shape[0], 4)
        
        srcSample = np.array([src[randIndices[0]], src[randIndices[1]], src[randIndices[2]], src[randIndices[3]]])
        dstSample = np.array([dst[randIndices[0]], dst[randIndices[1]], dst[randIndices[2]], dst[randIndices[3]]])
        
        A = []
        for i in range(srcSample.shape[0]):
            A.append([srcSample[i][0], srcSample[i][1], srcSample[i][2], 0, 0, 0, -dstSample[i][0] * srcSample[i][0], -dstSample[i][0] * srcSample[i][1], -dstSample[i][0] * srcSample[i][2]])
            A.append([0, 0, 0, srcSample[i][0], srcSample[i][1], srcSample[i][2], -dstSample[i][1] * srcSample[i][0], -dstSample[i][1] * srcSample[i][1], -dstSample[i][1] * srcSample[i][2]])

        A = np.array(A)
        AAT = A.T @ A
        w, v = np.linalg.eig(AAT)
        H = v[:, np.argmin(w)]
        H = H.reshape((3, 3))
        H = H / H[-1, -1]

        epsilon = 25
        count = 0
        inliersSrc = []
        inliersDst = []

        for k in range(src.shape[0]):
            x_ = H @ src[k, :].reshape(3, 1)
            x_ = x_[:2].reshape(1, 2)[0]
            x__ = (dst[k, 0:2].reshape(1, 2)[0] - x_)

            if abs(x__[0]) < epsilon and abs(x__[1]) < epsilon:
                count += 1
                inliersSrc.append(src[k, :])
                inliersDst.append(dst[k, :])

        if numInliers < count:
            numInliers = count
            inlierSrc = inliersSrc.copy()
            inlierDst = inliersDst.copy()

    inlierSrc = np.array(inlierSrc.copy())
    inlierDst = np.array(inlierDst.copy())
    srcSample = inlierSrc.copy()
    dstSample = inlierDst.copy()
    return srcSample, dstSample

# Function to calculate homography matrix
def calculateHomography(srcSample, dstSample):
    srcSample, dstSample = computeRANSACHomography(srcSample, dstSample)
    A = []
    for i in range(srcSample.shape[0]):
        A.append([srcSample[i][0], srcSample[i][1], srcSample[i][2], 0, 0, 0, -dstSample[i][0] * srcSample[i][0], -dstSample[i][0] * srcSample[i][1], -dstSample[i][0] * srcSample[i][2]])
        A.append([0, 0, 0, srcSample[i][0], srcSample[i][1], srcSample[i][2], -dstSample[i][1] * srcSample[i][0], -dstSample[i][1] * srcSample[i][1], -dstSample[i][1] * srcSample[i][2]])

    A = np.array(A)
    AAT = A.T @ A
    w, v = np.linalg.eig(AAT)
    H = v[:, np.argmin(w)]
    H = H.reshape((3, 3))
    H = H / H[-1, -1]
    return H

# Load images
image1 = cv.imread('image1.ppm')
image2 = cv.imread('image2.ppm')
image3 = cv.imread('image3.ppm')
image4 = cv.imread('image4.ppm')
image5 = cv.imread('image5.ppm')

# Compute SIFT feature matches and homography matrices
points1, points2 = computeSIFTMatches(image1, image2)
points1, points2 = computeRANSACHomography(points1, points2)
H1 = calculateHomography(points1, points2)

points3, points4 = computeSIFTMatches(image2, image3)
points3, points4 = computeRANSACHomography(points3, points4)
H2 = calculateHomography(points3, points4)

points5, points6 = computeSIFTMatches(image3, image4)
points5, points6 = computeRANSACHomography(points5, points6)
H3 = calculateHomography(points5, points6)

points7, points8 = computeSIFTMatches(image4, image5)
points7, points8 = computeRANSACHomography(points7, points8)
H4 = calculateHomography(points7, points8)

# Compute final homography
H = H4 @ H3 @ H2 @ H1

# Warp images
image1To5 = cv.warpPerspective(image1, H, (image5.shape[1], image5.shape[0]))
image1To5 = cv.cvtColor(image1To5, cv.COLOR_BGR2RGB)
image1 = cv.cvtColor(image1, cv.COLOR_BGR2RGB)
image5 = cv.cvtColor(image5, cv.COLOR_BGR2RGB)

print('Homography Matrix H = ', H)

givenHomographyInDataset = np.array([[6.2544644e-01, 5.7759174e-02, 2.2201217e+02],
                                     [2.2240536e-01, 1.1652147e+00, -2.5605611e+01],
                                     [4.9212545e-04, -3.6542424e-05, 1.0000000e+00]])

print('Sum of Squared Differences (SSD) between calculated H and given homography matrix: ', sum(sum((np.square(H - givenHomographyInDataset)))))

# Plot images
fig, ax = plt.subplots(1, 4, figsize=(20, 15))
image6 = cv.add(image5, image1To5)
ax[0].imshow(image1)
ax[0].set_title("Image 1")
ax[1].imshow(image5)
ax[1].set_title("Image 5")
ax[2].imshow(image1To5)
ax[2].set_title("Warped Image")
ax[3].imshow(image6)
ax[3].set_title("Stitched Image")

# Show the plot
plt.show()


AttributeError: module 'cv2' has no attribute 'xfeatures2d'